In [1]:
import os
from Acts.extractor import getCopora
from dotenv import load_dotenv

# load openAI key for inference
load_dotenv()
key = os.getenv('OPEN_AI_API')

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = key

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [8]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


docs = getCopora()[0:10]
len(docs)


10

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("What is corporation ?")